In [1]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 73.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 9.3 MB/s eta

In [2]:
import librosa
import numpy as np
from chromadb.config import Settings
from chromadb import Client

In [3]:
# Step 1: Function to extract audio embeddings using Mel-frequency cepstral coefficients (MFCC)
def extract_audio_embedding(audio_path):
    y, sr = librosa.load(audio_path, sr=None)  # Load audio file
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)  # Extract MFCC features
    embedding = np.mean(mfcc, axis=1)  # Take the mean across time frames
    return embedding

In [4]:
# Step 2: Initialize ChromaDB client
client = Client(Settings())
collection = client.get_or_create_collection(name="audio_collection")

In [5]:
# Step 3: Add audio files to the vector database
def add_audio_to_database(audio_paths):
    for idx, audio_path in enumerate(audio_paths):
        embedding = extract_audio_embedding(audio_path)
        metadata = {"path": audio_path}
        collection.add(
            ids=[f"audio_{idx}"],  # Changed 'embedding_ids' to 'ids'
            embeddings=[embedding],
            metadatas=[metadata]
        )


In [6]:
# Step 4: Perform similarity search for an audio query
def search_similar_audio(query_audio_path, top_k=3):
    query_embedding = extract_audio_embedding(query_audio_path)
    results = collection.query(query_embeddings=[query_embedding], n_results=top_k)
    # Accessing the list of metadatas correctly:
    for result_metadata in results["metadatas"]:  # Iterate through the list of metadatas
        print("Similar Audio Found:", result_metadata[0]["path"])  # Access the path from the first element

In [9]:
# Step 5: Example usage
if __name__ == "__main__":
    # Example audio files (replace with your actual audio file paths)
    audio_files = [
        "/content/sample_data/Recording.mp3",  # Replace with real audio file paths
        "/content/sample_data/Recording1.mp3",
        "/content/sample_data/Recording2.mp3"
    ]

    # Add audio files to the database
    add_audio_to_database(audio_files)

    # Query with a new audio file
    query_audio = "/content/sample_data/audio-file-sample.mp3"  # Replace with the query audio file path
    search_similar_audio(query_audio, top_k=3)


/tmp/ipython-input-1945749732.py:3: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_path, sr=None)  # Load audio file
/tmp/ipython-input-1945749732.py:3: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_path, sr=None)  # Load audio file
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/tmp/ipython-input-1945749732.py:3: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_path, sr=None)  # Load audio file
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, durat

FileNotFoundError: [Errno 2] No such file or directory: '/content/sample_data/audio-file-sample.mp3'